## Getting all messages from the database and applying sentiment analysis

In [17]:
import sys
sys.path.append('../sentiment_analysis')

# This is another way of doing the above, not sure which is better
sys.path.insert(0, '../mining')

from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from vader_sentiment_analysis import SentimentAnalyser
import pandas as pd

import connector
import query

In [13]:
df = pd.read_csv("../data/SELECT_ALL_FROM_all_messages.csv", header = 0)

In [14]:
len(df)

1354178

In [16]:
list(df)

['Unnamed: 0',
 'user_id',
 'contact_id',
 'relationship',
 'channel',
 'sent_at',
 'message']

In [8]:
df_meta = pd.read_csv("../data/SELECT_ALL_FROM_all_messages_metadata.csv", header = 0)

In [9]:
len(df_meta)

1070395

In [4]:
# Not able to read this in. 
#df = pd.read_csv("../data/all_messages_cleaned.csv", encoding='utf8', engine='python', usecols = ['user_id'], header = 0)

In [10]:
df = df.drop(['Unnamed: 0'],axis=1)

In [18]:
import psycopg2
from sshtunnel import SSHTunnelForwarder

port = 5432
remote_bind_address = ('localhost', 5432)
local_bind_address = ('localhost', 5432)
user_name = 'analytics'
dbname = 'dreikanter_production'
remote_host = 'dreikanter.production.devguru.co'
remote_port = 22
conn = None
server = None


params = {
            'database': 'dreikanter_production',
            'user': 'analytics',
            'host': 'localhost',
            'port': 5432
        }

server = SSHTunnelForwarder((remote_host, remote_port),ssh_username=user_name,
                                    local_bind_address=local_bind_address,
                                    remote_bind_address=remote_bind_address)
server.start()
print "We are in!"

#"dbname=dreikanter_production user=analytics host = localhost port = 5432"
conn = psycopg2.connect(**params)

We are in!


In [20]:
conn.close()
server.stop()

In [21]:

sql_query = """SELECT * from all_msgs_from_contacts_users"""

dbconnection = connector.ConnectDB()
conn = dbconnection.get_connection()
q = query.Query(conn, sql_query)
query_list = q.get_query_list()
query_df = q.get_query_dataframe()
#q.write_df_to_csv()
query_df.shape

(2867425, 6)

In [22]:
list(query_df)

['user_id', 'contact_id', 'relationship', 'channel', 'sent_at', 'message']

In [32]:
#["---------%s"%x for x in query_df['message'][0:20]]

In [33]:
q.write_df_to_csv()